# Classification
### Step 1: resize/crop images
Requires having run: `beescrape.py`
Depends on yolov5 and `shutil`

In [8]:
import os
from os import listdir
from shutil import copyfile
import shutil
from PIL import Image

# Directories
DATA_DIR = '/home/wilber/Documents/RESEARCH/research/beespotter'
OUT_DIR = '/m2docs/res/data'
CROPPED_PATH = '/m2docs/res/cropped_imgs'
cropped_files = listdir(CROPPED_PATH)

%rm -R /m2docs/res/data/*

# Args:
VAL_SIZE = 100
TEST_SIZE = 100
rotate = True
resize = True
size = (128, 128)

# Species with >= 1000 images:
classes = ['Apis_mellifera','Bombus_impatiens','Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']

total_skipped = 0
for label in classes:  # for each type of bee
    skipnum = 0
    classID = classes.index(label)
    
    input_path = os.path.join(DATA_DIR, label)
    
    # Attempt to create directories:
    if not os.path.exists(OUT_DIR + '/train/' + label):
        os.makedirs(OUT_DIR + '/train/' + label)
    if not os.path.exists(OUT_DIR + '/test/' + label):
        os.makedirs(OUT_DIR + '/test/' + label)
    if not os.path.exists(OUT_DIR + '/validation/' + label):
        os.makedirs(OUT_DIR + '/validation/' + label)
        
    train_path = os.path.join(OUT_DIR,'train/'+ label + '/')
    test_path  = os.path.join(OUT_DIR,'test/' + label + '/')
    valid_path = os.path.join(OUT_DIR,'validation/' + label + '/')
    
    index = 0
    im_list = os.listdir(input_path)
    for img in im_list:
        if img in cropped_files:
            pic = Image.open(os.path.join(CROPPED_PATH,img))
            
            if resize: #resize image
                out = pic.resize(size)
            else:
                out = pic
            
            if index < VAL_SIZE:
                out.save(os.path.join(valid_path,img))
            elif index < VAL_SIZE + TEST_SIZE:
                out.save(os.path.join(test_path,img))
            else:    #training set, rotate
                out.save(os.path.join(train_path,img))
                if rotate:
                    out.rotate(90).save(os.path.join(train_path,img.replace('.jpg','-90.jpg')))
                    out.rotate(180).save(os.path.join(train_path,img.replace('.jpg','-180.jpg')))
                    out.rotate(270).save(os.path.join(train_path,img.replace('.jpg','-270.jpg')))
        else:
            skipnum += 1
            index -= 1
            pass
        index += 1
    print(label, "skipped", skipnum)
    print(label, "total", index)
    
    total_skipped += errnum
    
print("Images moved. Skipped ", total_skipped)

Apis_mellifera skipped 2285
Apis_mellifera total 472
Bombus_impatiens skipped 2238
Bombus_impatiens total 1075
Bombus_auricomus skipped 751
Bombus_auricomus total 442
Bombus_bimaculatus skipped 1103
Bombus_bimaculatus total 637
Bombus_griseocollis skipped 1931
Bombus_griseocollis total 1011
Images moved. Skipped  41540


## Train classifier
Based on [this](https://nextjournal.com/gkoehler/pytorch-mnist) pytorch tutorial

Load images and train classifier.

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import datasets, transforms

classes = ['Apis_mellifera','Bombus_impatiens','Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']

ABS_PATH_TRAIN = '/m2docs/res/train'
ABS_PATH_VALID = '/m2docs/res/valid'

def load(dir_name, batch_size, shuffle = False):
	return(
        #create a data loader
        torch.utils.data.DataLoader(
            datasets.ImageFolder(root = dir_name, transform = transforms.ToTensor()),
            batch_size = batch_size,
            num_workers = 2,
            shuffle = shuffle
        )
    )

def train(max_epochs = 1, train_path = ABS_PATH_TRAIN, valid = ABS_PATH_VALID, labels = classes, batch_size = 24):
    length = 128
    epochs = max_epochs
    num_batches = batch_size
    class_names = labels
    num_classes = class_names.length

    torch.manual_seed(417)
    
    training_loader = load(train_path, batch_size_train, shuffle=True)

    return acc, epoch, stats
    
acc, epoch, stats = train()